In [1]:
from netCDF4 import Dataset
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt
import glob,os,pdb
import datetime as dt
%matplotlib ipympl
plt.close('all')

In [2]:
def calculate_bad_pixels(mean_dark,std_dark,bp=None):
    dead = np.zeros(mean_dark.shape)
    hot = dead.copy()
    cold = dead.copy()
    high_std = dead.copy()
    if bp is None:
        bp = np.zeros(dead.shape)
    dead[np.logical_and(std_dark < 1,bp<1)] = 1  # Dead Pixels
    dead[std_dark.mask] = 0
    
    cold[np.logical_and(mean_dark < 1000, std_dark < 1,bp<1)] = 1  # COLD Pixels
    cold[std_dark.mask] = 0
    
    hot[np.logical_and(mean_dark > 3000, std_dark < 1,bp<1)] = 1  # HOT Pixels
    hot[std_dark.mask] = 0
    
    high_std[np.logical_and(std_dark > 5 * np.nanmean(std_dark),bp<1)] = 1
    high_std[std_dark.mask] = 0

    return {'dead':dead[:],'cold':cold[:],'hot':hot[:],'noisy':high_std[:]}
    

In [3]:
ch4_files = [str(fi) for fi in sorted(list(Path('./2024').rglob('*CH4*.nc')))]
ch4_files.extend([str(fi) for fi in sorted(list(Path('./2025').rglob('*CH4*.nc')))])
o2_files = [str(fi) for fi in sorted(list(Path('./2024').rglob('*O2*.nc')))]
o2_files.extend([str(fi) for fi in sorted(list(Path('./2025').rglob('*O2*.nc')))])

ch4_files = np.array(ch4_files)
o2_files = np.array(o2_files)

ch4_bp = Dataset('../level1a_calibration_MSAT_20250722.0.0_CH4_BadPixelMap_CH4_20250722.nc','r')['BadPixelMap'][:] 
o2_bp = Dataset('../level1a_calibration_MSAT_20250722.0.0_O2_BadPixelMap_O2_20250722.nc','r')['BadPixelMap'][:] 


In [ ]:
plt.hist(ch4_bp.flatten(),bins=[0,1,3,5,11,24,50,100,150])

In [ ]:
plt.figure(); plt.pcolormesh(np.log(ch4_bp+0.000001)); plt.colorbar(); plt.show()

In [9]:
o2_files

array(['2024/06/14/00430001/po-1982/MethaneSAT_L0_O2_Dark_0_00430001.nc',
       '2024/06/15/00530001/202407311706/MethaneSAT_L0_O2_Dark_0_00530001.nc',
       '2024/06/21/00680001/po-1991/MethaneSAT_L0_O2_Dark_0_00680001.nc',
       '2024/06/21/00690001/po-1991/MethaneSAT_L0_O2_Dark_0_00690001.nc',
       '2024/06/21/006A0001/po-1991/MethaneSAT_L0_O2_Dark_0_006A0001.nc',
       '2024/06/22/00780001/po-2017/MethaneSAT_L0_O2_Dark_0_00780001.nc',
       '2024/06/22/00790001/po-2073/MethaneSAT_L0_O2_Dark_0_00790001.nc',
       '2024/07/19/00A30001/po-2065/MethaneSAT_L0_O2_Dark_0_00A30001.nc',
       '2024/07/19/00A30001/po-2067/MethaneSAT_L0_O2_Dark_0_00A30001.nc',
       '2024/07/19/00A40001/po-2065/MethaneSAT_L0_O2_Dark_0_00A40001.nc',
       '2024/07/19/00A40001/po-2067/MethaneSAT_L0_O2_Dark_0_00A40001.nc',
       '2024/07/19/00A50001/po-2065/MethaneSAT_L0_O2_Dark_0_00A50001.nc',
       '2024/07/19/00A50001/po-2067/MethaneSAT_L0_O2_Dark_0_00A50001.nc',
       '2024/07/19/00A60001/po-20

CH4:
CH4_00F20001
CH4_0A1E0001
CH4_063F0001
CH4_080F0001
CH4_05850001

O2:
O2_00F20001
O2_0A1E0001
O2_037E0001
O2_063F0001
O2_05850001

The following dark files have more bad pixels than usual and should be checked (and removed if necessary):
04A40001 07FA0001 05850001 091E0001 04900001 03BA0001 0A3E0001 05490001 
056F0001 06400001 03D10001 062B0001 0CA50001 05370001 06720001 0C750001 
06140001 04B00001 05220001 047C0001

## From David Miller:

Target 25 collect 046E0190
"gs://msat-prod-data-methanesat-level0-dark/0/2024/11/22/046A0001/202411291629_po-2247/MethaneSAT_L0_CH4_Dark_0_046A0001.nc"
"gs://msat-prod-data-methanesat-level0-dark/0/2024/11/22/046A0001/202411291629_po-2247/MethaneSAT_L0_O2_Dark_0_046A0001.nc"
Target 38 collect  01540260
"gs://msat-prod-data-methanesat-level0-dark/0/2024/09/12/014B0001/202409171319_po-2103/MethaneSAT_L0_CH4_Dark_0_014B0001.nc"
"gs://msat-prod-data-methanesat-level0-dark/0/2024/09/12/014B0001/202409171319_po-2103/MethaneSAT_L0_O2_Dark_0_014B0001.nc"
Target 192 collect  032F0C00
"gs://msat-prod-data-methanesat-level0-dark/0/2024/10/25/03290001/202411022044_po-2197/MethaneSAT_L0_CH4_Dark_0_03290001.nc"
"gs://msat-prod-data-methanesat-level0-dark/0/2024/10/25/03290001/202411022044_po-2197/MethaneSAT_L0_O2_Dark_0_03290001.nc"
Target 100 collect  0B100640
"gs://msat-prod-data-methanesat-level0-dark/0/2025/05/08/0AFD0001/202505101800_po-2524/MethaneSAT_L0_CH4_Dark_0_0AFD0001.nc""
"gs://msat-prod-data-methanesat-level0-dark/0/2025/05/08/0AFD0001/202505101800_po-2524/MethaneSAT_L0_O2_Dark_0_0AFD0001.nc"
Target 56 collect  09060380
"gs://msat-prod-data-methanesat-level0-dark/0/2025/03/20/09000001/202503212012_po-2455/MethaneSAT_L0_CH4_Dark_0_09000001.nc"
"gs://msat-prod-data-methanesat-level0-dark/0/2025/03/20/09000001/202503212012_po-2455/MethaneSAT_L0_O2_Dark_0_09000001.nc"


# Make a list of the suspect files

In [4]:
bad_pfx_list=sorted(list(set([
                    '0A1E','04A4','091E','037C','034E','0135', #037E
                    '0407','0135','04B0','03D1','0490','0A3E','056F',
                    '080F','0640','0537','063F','0585','03BA','00F2',
                    '07FA','0549','03D1','062B','0CA5','0537','0672',
                    '0C75','0614','04B0','0522','047C',
                    '046A','014B','0329','0AFD','0900'
                ])))
miller_list = sorted(['046A','014B','0329','0AFD','0900'])
bad_o2_files = {}
bad_ch4_files = {}
for pfx in miller_list:#bad_pfx_list:
    try:
        bad_o2_files[pfx] = o2_files[np.where([str(s).split('/')[3] == f'{pfx}0001' for s in o2_files])[0].flatten()][0]
        bad_ch4_files[pfx] = ch4_files[np.where([str(s).split('/')[3] == f'{pfx}0001' for s in ch4_files])[0].flatten()][0]
    except IndexError:
        continue

In [5]:
len(bad_o2_files)

5

# Plot all frames for each dark and save

In [6]:
os.chdir('/home/sean/sata/methanesat/darks/')
plt.close('all')
#bad_o2_files = {}
#bad_o2_file_list = open('noisy_o2_id','r').readlines()
#for ifi,fi in enumerate(bad_o2_file_list):
#    pfx = fi.split('/')[3]
#    bad_o2_files[pfx] = fi.strip('\n')
miller_list = ['046A','014B','0329','0AFD','0900']
for ip,pfx in enumerate(miller_list):#list(bad_pfx_list)):
    print(bad_o2_files[pfx])
    f = Dataset(str(bad_o2_files[pfx]))
    dn = f['Frame/PixelData'][:]
    n_rows = max(((dn.shape[0])//4+1,1))
    fig,axs=plt.subplots(n_rows,4,figsize=(10,10))
    for i in range(dn.shape[0]):
        if dn.shape[0] < 4:
            ax = axs[i]
        else:
            ax = axs[i//4,i%4]
        g = ax.pcolormesh(dn[i],vmin=1000,vmax=1500); 
        plt.colorbar(g,ax=ax)
        ax.set_title(f'{i}')
    fig.tight_layout()
    fig.suptitle(f'{pfx} Dark Map')
    fname = '_'.join(bad_o2_files[pfx].split('/')[:-1])+bad_o2_files[pfx].split('/')[-1][:-3]+'_o2_darks.png'
    fig.savefig(f'figs/all_frames/{fname}')
    plt.close('all')
    del dn

    f = Dataset(bad_ch4_files[pfx])
    dn = f['Frame/PixelData'][:]
    n_rows = max((dn.shape[0]//4+1,1))
    fig,axs=plt.subplots(n_rows,4,figsize=(10,10))
    for i in range(dn.shape[0]):
        if dn.shape[0] < 4:
            ax = axs[i]
        else:
            ax = axs[i//4,i%4]
        g = ax.pcolormesh(dn[i],vmin=1000,vmax=1500); 
        plt.colorbar(g,ax=ax)
        ax.set_title(f'{i}')
    fig.tight_layout()
    fig.suptitle(f'{pfx} Dark Map')
    fname = '_'.join(bad_ch4_files[pfx].split('/')[:-1])+bad_ch4_files[pfx].split('/')[-1][:-3]+'_ch4_darks.png'
    fig.savefig(f'figs/all_frames/{fname}')
    plt.close('all')
    del dn


2024/11/22/046A0001/202411252157_po-2247/MethaneSAT_L0_O2_Dark_0_046A0001.nc
2024/09/12/014B0001/202409171319_po-2103/MethaneSAT_L0_O2_Dark_0_014B0001.nc
2024/10/25/03290001/202411022044_po-2197/MethaneSAT_L0_O2_Dark_0_03290001.nc
2025/05/08/0AFD0001/202505101800_po-2524/MethaneSAT_L0_O2_Dark_0_0AFD0001.nc
2025/03/20/09000001/202503212012_po-2455/MethaneSAT_L0_O2_Dark_0_09000001.nc


In [ ]:
os.makedirs(pfx,exist_ok=True)

# Plot the mean and standard deviation of the DN and the high STD pixels

In [9]:
plt.close('all')
for pfx in miller_list:#list(bad_pfx_list):
    print(pfx)
    f = Dataset(bad_o2_files[pfx])
    dn = f['Frame/PixelData'][:]

    bp = calculate_bad_pixels(np.nanmean(dn,0),np.nanstd(dn,0),bp=o2_bp)
    high_std_bp = bp['noisy'][:]
    combined_bp = np.logical_or((high_std_bp > 0),(o2_bp > 0))

    fig,axs = plt.subplots(1,3,figsize=(8,2.7))
    g=axs[0].pcolormesh(dn.mean(0),vmin=1000,vmax=1500); plt.colorbar(g,ax=axs[0])
    g=axs[1].pcolormesh(dn.std(0),vmax=10); plt.colorbar(g,ax=axs[1])
    g=axs[2].pcolormesh(high_std_bp,vmin=0,vmax=1); plt.colorbar(g,ax=axs[2]);
    axs[0].set_title(f'Mean DN ({dn.shape[0]} Frm)')
    axs[1].set_title(f'Std DN')
    axs[2].set_title(f'STD BPM ({high_std_bp.sum()})')
    fig.suptitle(f'{pfx} 5*STD Bad Pixels')
    fig.tight_layout()
    fig.savefig(f'figs/bad_pixel_maps/{pfx}_O2_mean_std_bpmask.png')
    plt.close('all')

    f = Dataset(bad_ch4_files[pfx])
    dn = f['Frame/PixelData'][:]

    bp = calculate_bad_pixels(np.nanmean(dn,0),np.nanstd(dn,0),bp=ch4_bp)
    high_std_bp = bp['noisy'][:]
    combined_bp = np.logical_or((high_std_bp > 0),(ch4_bp > 0))

    fig,axs = plt.subplots(1,3,figsize=(8,2.7))
    g=axs[0].pcolormesh(dn.mean(0),vmin=1000,vmax=1500); plt.colorbar(g,ax=axs[0])
    g=axs[1].pcolormesh(dn.std(0),vmax=10); plt.colorbar(g,ax=axs[1])
    g=axs[2].pcolormesh(high_std_bp,vmin=0,vmax=1); plt.colorbar(g,ax=axs[2]);
    axs[0].set_title(f'Mean DN ({dn.shape[0]} Frm)')
    axs[1].set_title(f'Std DN')
    axs[2].set_title(f'STD BPM ({high_std_bp.sum()})')
    fig.suptitle(f'{pfx} 5*STD Bad Pixels')
    fig.tight_layout()
    fig.savefig(f'figs/bad_pixel_maps/{pfx}_CH4_mean_std_bpmask.png')
    plt.close('all')

046A
014B
0329
0AFD
0900
